In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

Reading Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/EFQM/session 5/Air_pollution.csv', index_col=['No'])
df.head()